In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gumbel_r

In [3]:
full=pd.read_csv('..\\..\\..\\Fermi-LAT Data\\hdu1.csv',index_col=0)
y_pred=pd.DataFrame()
y_proba=pd.DataFrame()

to_avg=pd.DataFrame()

for i in range(1,11):
    tempdf = pd.read_csv('Final Results\\Run '+str(i)+'\\y_results_BCU.csv',index_col=0)
    to_avg['y_proba_1_it'+str(i)] = tempdf['y_proba_1']
    y_pred['it'+str(i)]=tempdf['y_pred']

data=pd.DataFrame(to_avg.mean(axis=1),columns=['y_proba_1'],index=tempdf.index)

'''
 In 'data' all unknown BCUs are assigned 0. 
        All BLLs are assigned 1 and all FSRQs are assigned 2.
        Radio Galaxies = 3 and Other = 4

        In the machine learning algorithm '0' corresponds to a BLL and '1' corresponds 
        to an FSRQ.
'''

gumbel_vals = gumbel_r.pdf(data['y_proba_1'],0.3421238340282159,0.19413067816085203)
data['Fit Values']=gumbel_vals

rdgrank=pd.merge(data,full[['Source_Name','RAJ2000','DEJ2000','ASSOC1','ASSOC2']],how='inner',
         left_index=True,right_index=True)
rdgrank=rdgrank.sort_values('Fit Values',ascending=False)
rdgrank['Ranking']=np.arange(0,len(rdgrank),1)

rdgrank_ltx=pd.merge(data,full[['Source_Name','RAJ2000','DEJ2000','ASSOC1','ASSOC_PROB_BAY','ASSOC_PROB_LR']],
                     how='inner',left_index=True,right_index=True)
rdgrank_ltx=rdgrank_ltx.drop('y_proba_1',axis=1)
#rdgrank_ltx=rdgrank_ltx.drop('Fit Values',axis=1)
rdgrank_ltx=rdgrank_ltx.sort_values('Fit Values',ascending=False)
rdgrank_ltx=rdgrank_ltx.head(75)
rdgrank_ltx=rdgrank_ltx.drop('Fit Values',axis=1)
rdgrank_ltx.to_latex('latex_tables\\RDG_Ranked_top_75',index=False,float_format="{:.2f}".format,)
rdgrank.head()
err=to_avg.sem(axis=1)
err=err.sort_values()
print(err)

1483    0.006104
49      0.006161
541     0.006189
1477    0.006190
243     0.006224
          ...   
1412    0.007926
1046    0.008098
902     0.008174
1478    0.008508
1172    0.008562
Length: 1624, dtype: float64


: 

In [4]:
from astroquery.ipac.ned import Ned
from tqdm import tqdm
from astropy.table import Table
from astropy.io import fits,ascii

N=20

for i in tqdm(range(0,1)):
    ls=[1,7,8,11,12,16,23,29,32,36,44,48] # list of object with no associated counterparts
    ls2=[13,30,40,41,45] # list of objects with no NED data at all
    if i in ls:
        candidate = rdgrank['Source_Name'].loc[rdgrank['Ranking']==i]
    elif i in ls2:
        candidate = rdgrank['Source_Name'].loc[rdgrank['Ranking']==i]
        print(candidate)
        continue
    else:
        candidate = rdgrank['ASSOC1'].loc[rdgrank['Ranking']==i]
    fl=str(rdgrank['Source_Name'].loc[rdgrank['Ranking']==i])
    imageurl = Ned.get_image_list(candidate)
    image_toplot=Ned.get_images(candidate)
    fl= [idx for idx in fl.split(' ')]
    fl=fl[4]+fl[5]
    fl= [idx for idx in fl.split('\n')]
    fl=fl[0]
    hdu1 = fits.open(imageurl[0])
    pltscale=hdu1[0].header['PLTSCALE']
    xpixelsize=hdu1[0].header['XPIXELSZ']
    xpixelsizemm=xpixelsize*0.001
    pixelscale=pltscale*xpixelsizemm
    exposuretimeinminutes=hdu1[0].header['EXPOSURE']
    #print(exposuretime)
    for x in range(0,len(image_toplot)):
        image_data=image_toplot[0][x].data
        
        fig, ax = plt.subplots(figsize=(9,8),dpi=400)
        plt.title(f'Pixel Scale: {np.round(pixelscale,decimals=2)} arcsec/pixel',fontsize=N)
        plt.xticks(fontsize=N)
        plt.yticks(fontsize=N)
        plt.imshow(image_data, cmap='gray')
        cbar=plt.colorbar()
        cbar.ax.tick_params(labelsize=N)
        ax.tick_params('x',rotation=90)
        ax.set_xlabel('X (pixels)', fontsize=N)
        ax.set_ylabel('Y (pixels)', fontsize=N)
        cbar.set_label(f'No. Counts ({np.round(exposuretimeinminutes,decimals=2)} minute exposure)',
                        rotation=90,fontsize=N,labelpad=20)
        plt.subplots_adjust(bottom=0.2,left=0.1)
        plt.savefig(f'Images\\{str(fl)}_image_'+str(x)+'.png',dpi=300)
        #plt.show()
        plt.clf()
    #print(candidate)
#hdu1[0].header

100%|██████████| 1/1 [00:03<00:00,  3.04s/it]


<Figure size 3600x3200 with 0 Axes>

In [98]:
from astroquery.ipac.ned import Ned
import astropy.units as u
Ned.clear_cache()
newdf=pd.DataFrame(columns=['Name','Axis Ratio','Axis Ratio Uncertainty'])
    
for i in tqdm(range(18,19)):
    ls=[7,8,11,12,16,23,29,32,36,44,48] # list of object with no associated counterparts
    ls2=[1,2,4,5,7,8,13,30,40,41,45] # list of objects with no NED data at all
    if i in ls:
        candidate = rdgrank['Source_Name'].loc[rdgrank['Ranking']==i]
        print(candidate)
        continue
    elif i in ls2:
        candidate = rdgrank['Source_Name'].loc[rdgrank['Ranking']==i]
        print(candidate)
        continue
    else:
        candidate = rdgrank['ASSOC1'].loc[rdgrank['Ranking']==i]
    candidate=candidate.iloc[0]
    results=Ned.get_table(candidate)
    df=results.to_pandas()

    if 'Ned Axis Ratio' in list(df.columns):
        axisratio=df['NED Axis Ratio'].mean()
        axisunc=df['NED Axis Ratio Uncertainty'].mean()
        newdf.loc[i]=[candidate]+[axisratio]+[axisunc]
        #df['NED Axis Ratio Uncertainty']
    else:
        newdf.loc[i]=[candidate]+[np.nan]+[np.nan]
        
df.head()

100%|██████████| 1/1 [00:03<00:00,  3.40s/it]


,No.,Observed Passband,Photometry Measurement,Uncertainty,Units,Frequency,Flux Density,Upper limit of uncertainty,Lower limit of uncertainty,Upper limit of Flux Density,...,NED Uncertainty,NED Units,Refcode,Significance,Published frequency,Frequency Mode,Coordinates Targeted,Spatial Mode,Qualifiers,Comments
0,1,W1 (WISE),16.865,+/-0.062,mag,8.940000e+13,0.000056,0.000003,0.000003,NaN,...,+/-3.17E-06,Jy,2013wise.rept....1C,uncertainty,3.3526 microns,Broad-band measurement,,From fitting to map,Profile-fit,From new raw data
1,2,W1 (WISE),16.437,+/-0.106,mag,8.940000e+13,0.000082,0.000008,0.000008,NaN,...,+/-8.04E-06,Jy,2013wise.rept....1C,uncertainty,3.3526 microns,Broad-band measurement,,Flux in fixed aperture,"r=8.25"" COG-corrected",From new raw data; Uncorrected for known sourc...
2,3,W1 (WISE),15.484,+/-0.093,mag,8.940000e+13,0.000198,0.000017,0.000017,NaN,...,+/-1.70E-05,Jy,2013wise.rept....1C,uncertainty,3.3526 microns,Broad-band measurement,,Flux in fixed aperture,"r=22.0"" aperture",From new raw data; Uncorrected for known sourc...
3,4,W2 (WISE),16.232,+/-0.121,mag,6.510000e+13,0.000055,0.000006,0.000006,NaN,...,+/-6.16E-06,Jy,2013wise.rept....1C,uncertainty,4.6028 microns,Broad-band measurement,,From fitting to map,Profile-fit,From new raw data
4,5,W2 (WISE),15.703,+/-0.134,mag,6.510000e+13,0.000090,0.000011,0.000011,NaN,...,+/-1.11E-05,Jy,2013wise.rept....1C,uncertainty,4.6028 microns,Broad-band measurement,,Flux in fixed aperture,"r=8.25"" COG-corrected",From new raw data; Uncorrected for known sourc...
